In [1]:
import pandas as pd
import dask.dataframe as dd
import time
import psutil
import os
import matplotlib.pyplot as plt

In [6]:
#  تحديد مسار الملف
file_path = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv"  # ضع اسم أو مسار ملفك هنا

start_time = time.time()
process = psutil.Process(os.getpid())

chunk_size = 10_000
total_rows = 0

# قراءة الملف على دفعات
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    total_rows += len(chunk)
    # يمكنك تنفيذ أي معالجة على كل chunk هنا

mem_used = process.memory_info().rss / (1024 ** 2)  # بالميغابايت
end_time = time.time()

time_pandas = end_time - start_time
ram_pandas = mem_used

print(" Pandas-chunksize")
print(" الوقت:", round(time_pandas, 2), "ثانية")
print(" الذاكرة:", round(ram_pandas, 2), "MB")
print(" عدد الصفوف:", total_rows)


 Pandas-chunksize
 الوقت: 147.83 ثانية
 الذاكرة: 7364.03 MB
 عدد الصفوف: 67501979


In [3]:
start_time = time.time()
process = psutil.Process(os.getpid())

df_dask = dd.read_csv(file_path)
rows_dask = df_dask.shape[0].compute()

mem_used = process.memory_info().rss / (1024 ** 2)
end_time = time.time()

time_dask = end_time - start_time
ram_dask = mem_used

print(" Dask")
print(" الوقت:", round(time_dask, 2), "ثانية")
print(" الذاكرة:", round(ram_dask, 2), "MB")
print(" عدد الصفوف:", rows_dask)

 Dask
 الوقت: 96.55 ثانية
 الذاكرة: 1063.54 MB
 عدد الصفوف: 67501979


In [4]:
import gzip
import shutil

with open('/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv', 'rb') as f_in:
    with gzip.open('2019-Nov.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(" تم ضغط الملف بنجاح!")

 تم ضغط الملف بنجاح!


In [5]:
start_time = time.time()
process = psutil.Process(os.getpid())

df_compressed = pd.read_csv("2019-Nov.csv.gz", compression='gzip')

#  حساب الذاكرة المستعملة
mem_used = process.memory_info().rss / (1024 ** 2)

end_time = time.time()
time_compressed = end_time - start_time
ram_compressed = mem_used


print(" Pandas + GZIP (قراءة الملف الكامل)")
print(" الوقت:", round(time_compressed, 2), "ثانية")
print(" الذاكرة:", round(ram_compressed, 2), "MB")
print(" عدد الصفوف المقروءة:", len(df_compressed))

 Pandas + GZIP (قراءة الملف الكامل)
 الوقت: 220.19 ثانية
 الذاكرة: 7359.9 MB
 عدد الصفوف المقروءة: 67501979


In [7]:
results = pd.DataFrame({
    "Method": ["Pandas-chunksize", "Dask", "Pandas-gzip"],
    "Time (sec)": [time_pandas, time_dask, time_compressed],
    "Memory (MB)": [ram_pandas, ram_dask, ram_compressed]
})

print(results)

             Method  Time (sec)  Memory (MB)
0  Pandas-chunksize  147.833221  7364.027344
1              Dask   96.554926  1063.539062
2       Pandas-gzip  220.189628  7359.902344
